In [1]:
import json
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
from functools import lru_cache

In [2]:
# Set your Spotify app credentials
SPOTIPY_CLIENT_ID = '330a4427250d4e1aab13cb14859ead7b'
SPOTIPY_CLIENT_SECRET = '07bb4e0d8f12486482224ed60bd7da7a'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'  # e.g., 'http://localhost:8080'

In [3]:
# Authenticate with Spotify using OAuth
sp = Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope="user-library-read,user-top-read,user-read-recently-played"))

In [4]:
@lru_cache(maxsize=1)
def get_saved_tracks():
    saved_tracks = []
    tracks = sp.current_user_saved_tracks()
    while tracks:
        for i, item in enumerate(tracks['items']):
           saved_tracks.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return saved_tracks


In [6]:
saved_tracks = get_saved_tracks()

0 TOTO  -  Africa
1 Eminem  -  Houdini 2 Eminem  -  Brand New Dance 3 Eminem  -  Space Bound 4 NF  -  The Search 5 AJR  -  Burn the House Down 6 Marshmello  -  One Thing Right 7 Eminem  -  Beautiful 8 Eminem  -  Rap God 9 Macklemore  -  Glorious (feat. Skylar Grey) 10 Trevor Daniel  -  Falling
11 NF  -  PANDEMONIUM 12 NF  -  HAPPY 13 NF  -  Time 14 Eminem  -  When I'm Gone 15 Eminem  -  River (feat. Ed Sheeran) 16 *NSYNC  -  Bye Bye Bye - From Deadpool and Wolverine Soundtrack 17 Pitbull  -  Feel This Moment (feat. Christina Aguilera) 18 Maroon 5  -  This Love 19 Eagles  -  Hotel California - 2013 Remaster 0 Aerosmith  -  Dream On
1 KISS  -  I Was Made For Lovin' You 2 Hatikva 6  -  גיבורי על 3 Depeche Mode  -  Strangelove 4 Twenty One Pilots  -  Leave the City 5 Pitbull  -  Time of Our Lives 6 Panic! At The Disco  -  House of Memories 7 The Beatles  -  All My Loving - Remastered 2009 8 Kygo  -  Firestone 9 Alok  -  Deep Down (feat. Never Dull) 10 Martin Solveig  -  Do It Right
11 Scor

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [ ]:
for track in saved_tracks:
    print(track['track']['name'], end="\n")

In [49]:
top_tracks = sp.current_user_top_tracks(time_range='short_term')

In [ ]:
for item in top_tracks['items']:
    print(json.dumps(item))

In [12]:
@lru_cache(maxsize=1)
def get_recently_played(limit: int = 50, before: str = None, after: str = None):
    recently_played = []
    tracks = sp.current_user_recently_played(limit=limit, after=after, before=before)
    while tracks:
        for i, item in enumerate(tracks['items']):
           recently_played.append(item)
           track = item['track']
           if i % 10 == 0:
               print(i, track['artists'][0]['name'], ' - ', track['name'], end="\n")
           else:
              print(i, track['artists'][0]['name'], ' - ', track['name'], end=" ")
        
        if tracks['next']:
            tracks = sp.next(tracks)
        else:
            tracks = None
    return recently_played

In [13]:
before_unix_time = "1727038800"
recently_played = get_recently_played(after=before_unix_time)

0 Macklemore & Ryan Lewis  -  Can't Hold Us (feat. Ray Dalton)
1 Redbone  -  Come and Get Your Love - Single Version 2 Train  -  Drive By 3 TOTO  -  Africa 4 Wham!  -  Wake Me Up Before You Go-Go 5 Queen  -  Crazy Little Thing Called Love - Remastered 2011 6 Eminem  -  Houdini 7 Eminem  -  Trouble 8 Eminem  -  All You Got - skit 9 Eminem  -  Habits 10 Eminem  -  Brand New Dance
11 Imagine Dragons  -  Whatever It Takes 12 Eminem  -  The Monster 13 NF  -  Let You Down 14 NF  -  Let You Down 15 Confetti  -  Ghost 16 Twenty One Pilots  -  Heathens 17 Eminem  -  River (feat. Ed Sheeran) 18 Eminem  -  River (feat. Ed Sheeran) 19 Eminem  -  Road Rage 20 Eminem  -  Somebody Save Me
21 Eminem  -  Trouble 22 Eminem  -  Brand New Dance 23 Eminem  -  Fuel 24 Eminem  -  Kyrie & Luka 25 Eminem  -  Antichrist 26 Eminem  -  Habits 27 Eminem  -  Renaissance 28 Eminem  -  Guilty Conscience 2 29 Eminem  -  Head Honcho 30 Eminem  -  Breaking News - skit
31 Eminem  -  Tobey (feat. Big Sean and BabyTron) 32

In [14]:
recently_played

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3JhNCzhSMTxs9WLGJJxWOY'},
      'href': 'https://api.spotify.com/v1/artists/3JhNCzhSMTxs9WLGJJxWOY',
      'id': '3JhNCzhSMTxs9WLGJJxWOY',
      'name': 'Macklemore',
      'type': 'artist',
      'uri': 'spotify:artist:3JhNCzhSMTxs9WLGJJxWOY'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/4myTppRgh0rojLxx8RycOp'},
      'href': 'https://api.spotify.com/v1/artists/4myTppRgh0rojLxx8RycOp',
      'id': '4myTppRgh0rojLxx8RycOp',
      'name': 'Ryan Lewis',
      'type': 'artist',
      'uri': 'spotify:artist:4myTppRgh0rojLxx8RycOp'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/5BcAKTbp20cv7tC5VqPFoC'},
      'href': 'https://api.spotify.com/v1/artists/5BcAKTbp20cv7tC5VqPFoC',
      'id': '5BcAKTbp20cv7tC5VqPFoC',
      'name': 'Macklemore & Ryan Lewis',
      'type': 'artist',
      'uri': 'spotify:artist:5BcAKTbp20cv7tC